# Big Data Content Analytics - AUEB

## Introduction to Convolutional Networks for Image classification

* Lab Assistant: George Perakis
* Email: gperakis[at]aeub.gr | perakisgeorgios[at]gmail.com

### Importing Modules

In [ ]:
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.datasets import mnist

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras.losses import categorical_crossentropy
from tensorflow.python.keras import backend as K
from tensorflow.keras.optimizers import Adadelta, Adam

from typing import Tuple, List, Dict
import matplotlib.pyplot as plt

# %matplotlib notebook
# %pylab inline

%matplotlib inline

### Setting experiment hyperparameters

In [ ]:
# Input image dimensions
img_rows, img_cols = 28, 28

### Loading MNIST dataset

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')

In [ ]:
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
def reshape_img_input(x: np.ndarray,
                      img_rows: int = 28,
                      img_cols: int = 28,
                      normalize: bool = True) -> Tuple:
    """
    This function reshapes a n-dimensional numpy array of images into another format.
    Also normalizes the images
    
    :param x: N-dimensional array containing images
    :param img_rows: The output width of each of the images
    :param img_cols: The output height of each of the images
    :param normalize: Whether to normalize the images or not
    :return: A numpy array with the transformed images and the shape of each observation (image)
    """
    
    print(f'Original shape of nd-array: {x.shape}')
    
    if K.image_data_format() == 'channels_first':
        
        # the channel dimension goes to the front
        x1 = x.reshape(x.shape[0], 1, img_rows, img_cols)

        input_shape = (1, img_rows, img_cols)

    else:
        # the channel dimension goes to the end. So we end up with the following 4-D tensor
        # (N-samples, Height, Width, N-channels)
        x1 = x.reshape(x.shape[0], img_rows, img_cols, 1)

        input_shape = (img_rows, img_cols, 1)

    x1 = x1.astype('float32')

    if normalize:
        x1 /= 255
    
    
    return x1, input_shape

In [ ]:
x_train, input_shape = reshape_img_input(x=x_train,
                                         img_rows=28, 
                                         img_cols=28,
                                         normalize=True)

In [ ]:
x_test, _ = reshape_img_input(x=x_test,
                              img_rows=28, 
                              img_cols=28,
                              normalize=True)

In [ ]:
print(f'Input Shape: {input_shape}')

In [ ]:
print(f'x_train shape: {x_train.shape}')

print(f'{x_train.shape[0]} train samples')
print(f'{x_test.shape[0]} test samples')

### Conversion of the Y labels to One-Hot-Encoding

In [ ]:
# pre-proccessing parameters
num_classes = 10

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)

y_test = to_categorical(y_test, num_classes)

In [ ]:
# checking the difference in the shape
print('y_train shape: {}'.format(y_train.shape))
print('y_test shape: {}'.format(y_test.shape))

In [ ]:
y_train.sum(axis=1)

In [ ]:
# summing for each column (axis=0) to check the number of occurences of each digit.
# dividing with the total sum, in order to find the ratios

ratios = y_train.sum(axis=0) / y_train.sum()

for digit, value in enumerate(ratios):
    print("Digit: {} | Label Ratio: {:.2f} %".format(digit, 100 * value))

In [ ]:
def plot_digits_examples(x, y):
    """
    This function plots examples of digits (the first 9)
    
    :param x: the nd-array of the images
    :param y: the labels of the images
    :return: None
    """
    
    # creating a figure for all the digits that will be plotted
    fig1 = plt.figure()
    
    print(list(range(10)))
    print('-' * 31)
    
    for i in range(9):
        # create a subplot
        ax = fig1.add_subplot(191 + i)
        ax.clear()
        # plot the actual digit
        ax.imshow(x[i].reshape(28, 28), cmap='gray')
        
        print(y[i])
    plt.show()

In [ ]:
plot_digits_examples(x_train, y_train)

In [ ]:
plot_digits_examples(x_test, y_test)

## How Convolutions Work

<img src="https://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif">

## What are Strides and Padding

<img src="https://theano-pymc.readthedocs.io/en/latest/_images/numerical_padding_strides.gif">

## How Max Pooling Works

<img src="http://cs231n.github.io/assets/cnn/maxpool.jpeg">

## Build CNN Model

### Set hyperparameters

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping

In [ ]:
# Setting model hyperparameters
batch_size = 1024

epochs = 500

### Create Model Structure

In [ ]:
# Creating a sequential model
model = Sequential()

# adding a 2D Convolutional layer with 32 neurons with kernel size of 3X3, with relu activation.
model.add(Conv2D(32,
                 kernel_size=(3, 3), 
                 activation='relu',
                 input_shape=input_shape))

# adding a 2D Convolutional layer with 64 neurons with kernel size of 3X3, with relu activation.
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))

# Adding a Max Pooling layer to cut the size of the conv-layers 
model.add(MaxPooling2D(pool_size=(2, 2)))

# Adding dropout to regularize the model
model.add(Dropout(0.25))

# Flatten the MaxPooling Layer.
model.add(Flatten())

# Adding another Dense layer of 128 neurons with Relu activation.
model.add(Dense(128, activation='relu'))

# Adding dropout to regularize the model
model.add(Dropout(0.4))

# We have a MULTI-CLASS problem. This is the reason we use 10 neurons (as many as the digits)
# with the SOFTMAX activation. 
model.add(Dense(num_classes,
                activation='softmax'))

print(model.summary())

In [ ]:
# How to plot a nice neural net represenation using LaTeX
# https://github.com/HarisIqbal88/PlotNeuralNet

### Model compilation

In [ ]:
# Remember, we have a multi-class problem to solve with 10 classes
# that's why we will use the categorical_crossentropy as a loss function. 

# Also, introducing another variation of the optimizers, the Adadelta().
# You could also use the Adam as well. 
model.compile(
    loss=categorical_crossentropy,
    optimizer=Adadelta(),
    metrics=['accuracy'])

### Setting Callbacks

In [ ]:
# help(keras.callbacks.EarlyStopping)

In [ ]:
# early stopping callback

es = EarlyStopping(
    monitor   = 'val_loss', # which metric we want to use as criterion to stop training
    min_delta = 0, # Minimum change in the monitored quantity to qualify as an improvement
    patience  = 3, # we 3 epochs before stopping
    verbose   = 1, # verbosity level
    mode      = 'auto',
    restore_best_weights = True)

### Fitting the Model

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size = batch_size,
    epochs = epochs,
    verbose = 1,
    validation_split = 0.1,
    callbacks = [es])

### Model Evaluation

In [ ]:
score = model.evaluate(x_test,
                       y_test,
                       verbose=1)

print(f'Test loss: {score[0]}:')
print('Test accuracy: {:.3f} %'.format(100 * score[1]))

In [ ]:
y_test_pred = model.predict_classes(x_test)

In [ ]:
y_test_pred

In [ ]:
y_test.argmax(axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [ ]:
conf_mat = confusion_matrix(y_test.argmax(axis=1),
                            y_test_pred)

pd.DataFrame(conf_mat)

In [ ]:
print(classification_report(y_test.argmax(axis=1),
                            y_test_pred,
                            digits=4))